In [1]:
"""
Feature: spinal_cord_injury
Extract from diagnoses_icd using ICD codes
"""
import pandas as pd
import os
from config_paths import *
from utils import drop_columns

In [2]:
# Read dataset
df = pd.read_csv(dataset_path)
initial_row_count = len(df)

# Load diagnoses
diagnoses = pd.read_csv(
    os.path.join(hosp_path, "diagnoses_icd.csv"),
    usecols=["subject_id", "hadm_id", "icd_code", "icd_version"]
)
diagnoses["icd_code"] = diagnoses["icd_code"].astype(str)

# Function to check if any ICD code matches patterns
def has_icd(icd_list, patterns):
    return icd_list.str.startswith(tuple(patterns)).any()

# Spinal cord injury patterns
spinal_cord_injury_patterns = [
    "806",     # Fracture of vertebral column with spinal cord injury
    "952",     # Spinal cord lesion without vertebral fracture
    "3440",    # Quadriplegia
    "3441",    # Paraplegia
    "336",     # Other acute spinal cord diseases
    "9072",    # Late effect of spinal cord injury
    "7674",    # Birth injury to spine/spinal cord
    "76872",   # Birth asphyxia spinal related
    "S140", "S141",   # Cervical SCI
    "S240", "S241",   # Thoracic SCI
    "S340", "S341",   # Lumbar SCI
    "G82",            # Paralysis (para-/quadri-/tetraplegia)
    "G830", "G831", "G832", "G833", "G834", "G839",  # Other SCI syndromes
    "T060", "T061",   # Extensive injuries involving spinal cord
    "T913"            # Sequelae of spinal cord injury
]

spinal_cord_injury_df = (
    diagnoses
    .groupby(["subject_id", "hadm_id"])
    .apply(
        lambda g: pd.Series({
            "spinal_cord_injury": has_icd(g["icd_code"], spinal_cord_injury_patterns)
        }),
        include_groups=False
    )
    .reset_index()
)

# Merge with dataset
df = df.merge(
    spinal_cord_injury_df,
    on=["subject_id", "hadm_id"],
    how="left"
)

# Fill missing values with False
df["spinal_cord_injury"] = df["spinal_cord_injury"].fillna(False).astype(bool)

# Save
df.to_csv(dataset_path, index=False)

# Diagnostic output
print(f"\nTotal rows: {len(df)}")
print(f"Expected total: {initial_row_count}")
if len(df) == initial_row_count:
    print(f"✓ Row count matches expected total ({initial_row_count})")
else:
    print(f"⚠ Row count mismatch! Expected {initial_row_count}, got {len(df)} (difference: {len(df) - initial_row_count})")
print(f"Dataset shape: {df.shape}")
print(f"Feature 'spinal_cord_injury' added. True count: {df['spinal_cord_injury'].sum()}")
print(f"Dataset shape: {df.shape}")




C:\Users\Coditas\AppData\Local\Temp\ipykernel_13844\2193778816.py:55: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["spinal_cord_injury"] = df["spinal_cord_injury"].fillna(False).astype(bool)



Total rows: 158020
Expected total: 158020
✓ Row count matches expected total (158020)
Dataset shape: (158020, 46)
Feature 'spinal_cord_injury' added. True count: 2878
Dataset shape: (158020, 46)


In [3]:
df.columns

Index(['subject_id', 'hadm_id', 'cauti_type', 'cauti_icd_codes',
       'other_uti_icd_codes', 'remaining_icd_codes', 'gender', 'anchor_age',
       'admittime', 'dischtime', 'admission_type', 'admission_location',
       'discharge_location', 'race', 'catheter_procedures',
       'catheter_procedures_ir', 'catheter_used_in_procedures_ir',
       'catheter_insertion_date_ir', 'catheter_removal_date_ir',
       'catheter_outputevents', 'catheter_used_in_output_events',
       'datetimeevents', 'catheter_used_in_datetime_events',
       'insertion_date_from_datetimeevents',
       'removal_date_from_datetimeevents', 'catheter_procedure_events',
       'catheter_used_in_procedure_events', 'catheter_size_from_chartevents',
       'index', 'catheter_clinical_notes', 'catheter_used_in_clinical_notes',
       'catheter_present', 'final_cauti_flag', 'final_insertion_date',
       'final_removal_date', 'catheter_duration_days', 'BMI_in_admission',
       'BMI_computed', 'BMI_last_year', 'BMI', 

In [5]:
# cols_to_drop = [
#  "spinal_cord_injury"
# ]
# df = drop_columns(df, cols_to_drop)
# df.to_csv(dataset_path, index=False)